In [2]:
import csv
import sqlite3

In [21]:
filename = 'Delta_DL_Aug2022_Marketing.csv'
faa_csv = csv.reader(open(filename, encoding='utf8'))
conn = sqlite3.connect('faa')
conn.execute('DROP TABLE IF EXISTS faa')    


for i, row in enumerate(faa_csv):
    if i == 0:
        colnames = ', '.join(row)
        sql = f'CREATE TABLE faa ({colnames});'
        conn.execute(sql)
    else:
        sql_base = f'''INSERT INTO faa VALUES ({('?,'*len(row))[:-1]})'''
        sql_data = tuple(row)
        conn.execute(sql_base, sql_data)
conn.commit()
conn.close()


In [65]:
conn = sqlite3.connect('faa')
origin = "'ATL'"
destination = "'DTW'"
flight_date = "'8/3/2022'"
tc = 60 # in minutes

sql = f'''SELECT
            start.Origin,
            start.Dest,
            start.Date,
            start.FlightNum,
            start.DepTime,
            start.ArrTime,
            finish.Origin,
            finish.Dest,
            finish.Date,
            finish.FlightNum,
            finish.DepTime,
            finish.ArrTime,
            CAST((finish.DepTime/100) AS int)*60 + finish.DepTime%100 -
                CAST((start.ArrTime/100) AS int)*60 - start.ArrTime%100 AS connectTime
          FROM 
              (SELECT
                  Origin,
                  Dest,
                  FlightDate AS Date,
                  Flight_Number_Operating_Airline AS FlightNum,
                  CAST(CRSDepTime AS int) AS DepTime,
                  CAST(CRSArrTime AS int) AS ArrTime
              FROM faa
              WHERE
                  Origin = {origin} AND
                  Dest <> {destination} AND
                  FlightDate = {flight_date}) AS start,
              (SELECT
                  Origin,
                  Dest,
                  FlightDate AS Date,
                  Flight_Number_Operating_Airline AS FlightNum,
                  CAST(CRSDepTime AS int) AS DepTime,
                  CAST(CRSArrTime AS int) AS ArrTime
              FROM faa
              WHERE
                  Origin <> {origin} AND
                  Dest = {destination} AND
                  FlightDate = {flight_date}) AS finish
        WHERE
            start.Dest = finish.Origin AND
            connectTime > {tc}
      '''

cursor = conn.execute(sql)
for c in cursor.fetchall():
    print(c)

('ATL', 'LEX', '8/3/2022', '5660', 910, 1018, 'LEX', 'DTW', '8/3/2022', '4932', 1530, 1648, 312)
('ATL', 'CHA', '8/3/2022', '5340', 814, 907, 'CHA', 'DTW', '8/3/2022', '4907', 1640, 1828, 453)
('ATL', 'LAX', '8/3/2022', '303', 1930, 2110, 'LAX', 'DTW', '8/3/2022', '1216', 2335, 702, 145)
('ATL', 'DCA', '8/3/2022', '306', 1035, 1219, 'DCA', 'DTW', '8/3/2022', '1637', 2040, 2219, 501)
('ATL', 'DCA', '8/3/2022', '306', 1035, 1219, 'DCA', 'DTW', '8/3/2022', '2995', 1736, 1915, 317)
('ATL', 'SEA', '8/3/2022', '334', 815, 1041, 'SEA', 'DTW', '8/3/2022', '1602', 2315, 630, 754)
('ATL', 'SEA', '8/3/2022', '334', 815, 1041, 'SEA', 'DTW', '8/3/2022', '2318', 1520, 2240, 279)
('ATL', 'SEA', '8/3/2022', '334', 815, 1041, 'SEA', 'DTW', '8/3/2022', '2958', 1215, 1938, 94)
('ATL', 'LAX', '8/3/2022', '335', 1635, 1815, 'LAX', 'DTW', '8/3/2022', '1216', 2335, 702, 320)
('ATL', 'LAX', '8/3/2022', '335', 1635, 1815, 'LAX', 'DTW', '8/3/2022', '2683', 2205, 527, 230)
('ATL', 'SEA', '8/3/2022', '339', 1630,